# Stock Forecast

## Environment

In [17]:
from pathlib import Path
from pandas import read_csv, to_datetime, cut, DataFrame, date_range

## Parameters

In [2]:
BASE_DIR = '/Users/efraflores/Desktop/EF/Corner/Catalog/Stock_fcst/'
FILE_NAME = 'op_9988.csv'
PRODUCT_ID_COL = 'product_id'
DATE_COL = 'actual_delivery_time'
REQUESTED_COL = 'quantity'
FOUND_COL = 'quantity_found'
TIMEWINDOWS = [-1,8,10,11,13,15,17,19,21,23]
VALIDATION_SIZE = 0.1

## Import

In [3]:
df = read_csv(Path(BASE_DIR).joinpath(FILE_NAME))
print(len(df))
display(df.sample())

220139


,actual_delivery_time,product_id,quantity,quantity_found
4974,2021-04-05T19:39:08.892734,2115229,1.5,2.645


## Validation split

In [4]:
df.sort_values(DATE_COL, inplace=True)
row_limit = int(len(df)*VALIDATION_SIZE)
val = df.iloc[:row_limit,:]
df = df.iloc[-row_limit:,:]

## Functions

### Timewindows

In [8]:
def set_timewindows(data, date_col, timewindows, datetime_format='%Y-%m-%dT%H:%M:%S'):
    df = data.copy()
    df[date_col] = to_datetime(df[date_col], format=datetime_format)
    df[['date','time']] = df[date_col].astype(str).str.split(expand=True)
    df['date'] = to_datetime(df['date'])
    df['time'] = df['time'].apply(lambda x: x[:8])
    df['timewindow'] = cut(df[date_col].dt.hour, bins=timewindows)
    df['timewindow'] = df['timewindow'].apply(lambda x: str(x.left+1).zfill(2)+':00 a '+str(x.right).zfill(2)+':59')
    timewindows = DataFrame(set(df['timewindow']),columns=['timewindow']).sort_values('timewindow').reset_index(drop=True)
    return df, timewindows

# test,timewindows = set_timewindows(df, DATE_COL, TIMEWINDOWS)
# print(timewindows)
# test.sample()

In [6]:
def qty_acum(data, id_col, requested_col, found_col, date_col='date', timewindow_col='timewindow'):
    df = data.copy()
    df = df.groupby([id_col, date_col, timewindow_col])[[requested_col, found_col]].sum().reset_index()
    #df['fr'] = df[found_col] / (df[requested_col]+1e-10)
    df.sort_values([id_col, date_col], inplace=True)
    for col in [requested_col, found_col]:
        df[f'acum_{col}'] = df.groupby([id_col, date_col])[col].cumsum()
    return df

# test,timewindows = set_timewindows(df, DATE_COL, TIMEWINDOWS)
# test = qty_acum(test, PRODUCT_ID_COL, REQUESTED_COL, FOUND_COL)
# test.head()

In [31]:
def full_dates(data, timewindows, id_col, date_col='date'):
    df = data.copy()
    total = DataFrame()
    df[date_col] = df[date_col].astype(str)
    for row in set(df[id_col]):
        #Set the id_col as index (again) to call all the rows with that id_col
        df_id = df.set_index(id_col).loc[row,:]
        #All possible dates from the min to the max of the subset
        tot_dates = DataFrame(date_range(start=df_id[date_col].min(), end=df_id[date_col].max()).date, columns=[date_col]).astype(str)
        #All possible timewindows for every date
        tot_tw = DataFrame()
        for date in tot_dates[date_col]:
            aux = DataFrame([(date,x) for x in timewindows['timewindow']], columns=[date_col,'timewindow'])
            tot_tw = tot_tw.append(aux)
        
        df_id = df_id.merge(tot_tw, on=[date_col,'timewindow'], how='right').fillna(0) ######## FILL THE VALUE ABOVE
        total = total.append(df_id)
    return total

test,timewindows = set_timewindows(df.sample(frac=.1), DATE_COL, TIMEWINDOWS)
test = qty_acum(test, PRODUCT_ID_COL, REQUESTED_COL, FOUND_COL)
test = full_dates(test, timewindows, PRODUCT_ID_COL)
test

,date,timewindow,quantity,quantity_found,acum_quantity,acum_quantity_found
0,2021-07-06,00:00 a 08:59,0.0,0.0,0.0,0.0
1,2021-07-06,09:00 a 10:59,0.0,0.0,0.0,0.0
2,2021-07-06,11:00 a 11:59,0.0,0.0,0.0,0.0
3,2021-07-06,12:00 a 13:59,0.0,0.0,0.0,0.0
4,2021-07-06,14:00 a 15:59,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...
112,2021-07-18,14:00 a 15:59,0.0,0.0,0.0,0.0
113,2021-07-18,16:00 a 17:59,0.0,0.0,0.0,0.0
114,2021-07-18,18:00 a 19:59,0.0,0.0,0.0,0.0
115,2021-07-18,20:00 a 21:59,0.0,0.0,0.0,0.0


In [32]:
test.isnull().sum()

date                   0
timewindow             0
quantity               0
quantity_found         0
acum_quantity          0
acum_quantity_found    0
dtype: int64

In [16]:
def multishift(data, id_cols, date_cols, shifts, **pivot_args):
    df = data.copy()
    #Merge all column names as a string
    id_col = ','.join(id_cols)
    #And as a column
    df[id_col] = df[id_cols].apply(lambda x:','.join(x.dropna().astype(str)),axis=1)
    #Drop any "id_col"-set that has a lower frequency than the max of the "shifts-list"
    # freq = df[id_col].value_counts().to_frame()
    # omit_idx = freq[freq[id_col]<=max(shifts)].index.to_list()
    # if len(omit_idx)>0:
    #     df = df[~df[id_col].isin(omit_idx)].copy()
    #Change data structure to build the "shifting"
    df = df.pivot_table(index=[id_col]+date_cols,
                        **pivot_args,
                        fill_value=0)
    #Concatenate multiple columns if they are
    df.columns = ['_'.join([x for x in col]) if 
                  not isinstance(df.columns[0],str) #First element is not a string
                  else col for col in df.columns]
    #Bring the id_col for taking the set (unique values) in the next loop
    df.reset_index(inplace=True)
    #Each shift must be calculated at "id_col" level
    total = DataFrame()
    for row in set(df[id_col]):
        #Set the id_col as index (again) to call all the rows with that id_col
        df_id = df.set_index(id_col).loc[row:row,:]
        cols = df_id.columns[len(date_cols):].tolist()
        #Start the "shifting"
        aux = df_id.copy().iloc[:,len(date_cols):].reset_index(drop=True)
        for i in shifts:
            shifted = df_id.iloc[:,len(date_cols):].shift(i).rename(columns={x:f'{x}_{str(i).zfill(2)}'
                                                                             for x in cols})
            aux = aux.join(shifted.reset_index(drop=True))
        aux[id_col] = row
        total = total.append(aux,ignore_index=True)
    total[[x for x in id_col.split(',')]] = total[id_col].str.split(',',expand=True)
    total[date_cols] = df[date_cols].copy()
    total.set_index(id_cols+date_cols, inplace=True)
    return total[[x for x in total.columns if x not in [id_col]]]

In [17]:
def model_structure(data, target_col):
    X = data[[x for x in data.columns if x not in [target_col]]]
    y = data[target_col].values
    return X,y

## Transform

In [ ]:
aux = set_timewindows(df, DATE_COL, TIMEWINDOWS)[0]
aux = qty_acum(aux.head(33),DATE_COL, ['product_id','date'], 'quantity', 'quantity_found')
aux

In [30]:
df,timewindows = set_timewindows(df, DATE_COL, TIMEWINDOWS)
timewindows

{'00:00 a 08:59',
 '09:00 a 10:59',
 '11:00 a 11:59',
 '12:00 a 13:59',
 '14:00 a 15:59',
 '16:00 a 17:59',
 '18:00 a 19:59',
 '20:00 a 21:59',
 '22:00 a 23:59'}

In [68]:
tw = DataFrame(timewindows,columns=['timewindow']).sort_values('timewindow')
df_aux = DataFrame(sset(aux['timewindow']),columns=['timewindow'])
df_aux['isin'] = 1
tw = tw.merge(df_aux, how='left')
first_tw = tw['isin'].first_valid_index()
last_tw = tw['isin'].last_valid_index()
tw['isin'].loc[first_tw:last_tw] = tw['isin'].loc[first_tw:last_tw].ffill()
tw.dropna().merge(aux, how='left')

/Users/efraflores/Desktop/hub/stock_forecast/stock_venv/lib/python3.9/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


,timewindow,isin,actual_delivery_time,product_id,quantity,quantity_found,date,time
0,11:00 a 11:59,1.0,2021-07-11 11:20:25.508505,1235786.0,0.300,0.555,2021-07-11,11:20:25
1,12:00 a 13:59,1.0,2021-07-18 12:01:06.129280,1579075.0,2.000,2.000,2021-07-18,12:01:06
2,12:00 a 13:59,1.0,2021-07-15 13:45:46.632610,2011111.0,0.396,0.350,2021-07-15,13:45:46
3,14:00 a 15:59,1.0,NaT,NaN,NaN,NaN,NaT,NaN
4,16:00 a 17:59,1.0,NaT,NaN,NaN,NaN,NaT,NaN
5,18:00 a 19:59,1.0,2021-07-18 18:01:13.027754,335747.0,1.000,1.000,2021-07-18,18:01:13


In [ ]:
df = found_rate(df, REQUESTED_COL, FOUND_COL)
shifted = multishift(df, id_cols=['product_id'], date_cols=['date','timewindow'], shifts=range(1,5), values='fr', aggfunc='mean')
X,y = model_structure(shifted, 'fr')

## Training

In [12]:
shifted.head(10)

fr  fr_01  fr_02  fr_03  fr_04
product_id date       timewindow                                    
117866     2021-07-12 18:00 a 19:59  1.0    NaN    NaN    NaN    NaN
435655     2021-07-10 00:00 a 08:59  1.0    NaN    NaN    NaN    NaN
           2021-07-17 11:00 a 11:59  1.0    1.0    NaN    NaN    NaN
1071278    2021-07-18 18:00 a 19:59  1.0    NaN    NaN    NaN    NaN
357342     2021-07-08 14:00 a 15:59  1.0    NaN    NaN    NaN    NaN
                      09:00 a 10:59  1.0    1.0    NaN    NaN    NaN
336765     2021-07-15 00:00 a 08:59  1.0    NaN    NaN    NaN    NaN
                      16:00 a 17:59  0.0    1.0    NaN    NaN    NaN
           2021-07-18 12:00 a 13:59  0.0    0.0    1.0    NaN    NaN
314647     2021-07-13 18:00 a 19:59  1.0    NaN    NaN    NaN    NaN

In [ ]:
################ MAKE THE SHIFTS CONSIDER ALL TIME WINDOWS (OR JUST FROM THE LOWEST TO THE HIGHEST)